### Test microservice

In [16]:
%load_ext autoreload
%autoreload 2

import json
import base64
import requests
from glob import glob


imgs = glob("./dataset/*.jpeg")
with open(imgs[0], 'rb') as image:
    image_read = image.read()
image_64_encode= base64.b64encode(image_read).decode('utf-8') #encodestring also works aswell as decodestring
        

data ={"data": { "ndarray": ['image_id',image_64_encode],
                "names": ['image_id','image']}}

headers = {"Content-Type": "application/json"}
json_response = requests.post('http://192.168.30.13:5004/api/v1.0/predictions', data=json.dumps(data), headers=headers)

json_response.json()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ConnectionError: HTTPConnectionPool(host='192.168.30.13', port=5004): Max retries exceeded with url: /api/v1.0/predictions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f4e8c1642b0>: Failed to establish a new connection: [Errno 111] Connection refused',))

### Serving the microservice on Kubernetes with Seldon Deployment


**install Seldon Operator**

```bash
helm install seldon-core seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --set istio.enabled=true --set usageMetrics.enabled=true --namespace seldon-system
```


In [17]:
%load_ext autoreload
%autoreload 2

import json
import base64
import requests
from glob import glob


imgs = glob("./dataset/*.jpeg")
with open(imgs[0], 'rb') as image:
    image_read = image.read()
image_64_encode= base64.b64encode(image_read).decode('utf-8') #encodestring also works aswell as decodestring
        


data ={"data": { "ndarray": ['image_id',image_64_encode],
                "names": ['image_id','image']}}

headers = {"Content-Type": "application/json"}
json_response = requests.post('http://192.168.30.13/seldon/default/checks-reader/api/v1.0/predictions', data=json.dumps(data), headers=headers)
json_response.json()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ConnectionError: HTTPConnectionPool(host='192.168.30.13', port=80): Max retries exceeded with url: /seldon/default/checks-reader/api/v1.0/predictions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f4e8c164908>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [12]:
#Test with gRPC
from client import timer
# test with any input message
#grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0]]}}'  -rpc-header seldon:grpc-seldon -rpc-header namespace:seldon     -plaintext     -proto ./prediction.proto  0.0.0.0:5000 seldon.protos.Seldon/Predict

with open('data.json', 'w') as f:
    json.dump(data, f)
    
with timer("grpc"):
    output = !cd proto && cat ../data.json |  grpcurl -v -d @  -rpc-header seldon:grpc-seldon -rpc-header namespace:seldon -max-time 20 -max-msg-sz 16777216    -plaintext          -proto ./prediction.proto  0.0.0.0:5000 seldon.protos.Seldon/Predict

output

[grpc] done in 0 sec.


['/bin/bash: grpcurl: command not found']